# Cleaning Data

_In order for something to become clean, something else must become dirty._ 

Imbesi's Law of the Conservation of Filth

A speaker at a data science conference a few years ago made a joke—perhaps one already widely repeated by the time he told it—about talking with a colleague of his.  The colleague complained of data cleaning taking up half of her time, in response to which the speaker expressed astonishment that it could be so little as 50%.

Without worrying too much about assigning a precise percentage, in my experience working as a technologist and data scientist, I have found that the bulk of what I do is preparing my data for the statistical analyses, machine learning models, or nuanced visualizations that I would like to utilize it for.  Although hopeful executives, or technical managers a bit removed from the daily work, tend to have an eternal optimism that the next set of data the organization acquires will be clean and easy to work with, I have yet to find that to be true in my concrete experience.  

## Doing the Other 80% of the Work

It is something of a truism in data science, data analysis, or machine learning that most of the effort needed to achieve your actual purpose lies in cleaning your data.

Certainly, some data is better and some is worse.  But *all data is dirty*, at least within a very small margin of error in the tally.  Even data sets that have been published, carefully studied, and that are widely distributed as canonical examples for statistics textbooks or software libraries, generally have a moderate number of data integrity problems.  Even after our best pre-processing, a more attainable goal should be to make our data *less dirty*; making it *clean* remains unduly utopian in aspiration.

By all means we should distinguish *data quality* from *data utility*.  These descriptions are roughly orthogonal to each other.  Data can be dirty (up to a point) but still be enormously useful.  Data can be (relatively) clean but have little purpose, or at least not be fit for purpose.  Concerns about the choice of measurements to collect, or about possible selection bias, or other methodological or scientific questions are mostly outside the scope of this book.  However, a fair number of techniques I present can *aid* in evaluating the utility of data, but there is often no mechanical method of remedying systemic issues.  For example, statistics and other analyses may reveal—or at least strongly suggest—the unreliability of a certain data field.  But the techniques in this book cannot generally automatically fix that unreliable data or collect better data.

The code shown throughout this book is freely available.  However, the purpose of this book is not learning to use the particular tools used for illustration, but to understand the underlying purpose of data quality.  The concepts presented should be applicable in any programming language used for data processing and machine learning.  I hope it will be easy to adapt the techniques I show to your own favorite collection of tools and programming languages.

## Types of Grime

There are two kinds of problems in data sets
* Structural problems in the formatting of data
* content problems in the actual values recorded.

A format might "record values in the wrong place or of the wrong type."  Sometimes well formatted data is implausible or wrong because of flawed instruments, transcription errors, numeric overflows, or via other pitfalls.

The first lessons discussing "data ingestion" are more focused on structural problems.  It is not always cleanly possible to separate these issues, but we begin with an emphasis on format problems. In later lessons, we look at anomalies, data quality, feature engineering, value imputation, and model-based cleaning to direct attention to content issues.

In the case of structural problems, we almost always need manual remediation of the data.  Exactly where the bytes that make up the data go wrong can vary enormously, and usually does not follow a pattern that lends itself to a single high-level description.  Often we have a somewhat easier time with the content problems, but at the same time they are more likely to be irremediable even with manual work.

Consider a small raw data set about a school class.

In [1]:
from src.intro_students import data, cleaned
print(data)


Student#,Last Name,First Name,Favorite Color,Age
1,Johnson,Mia,periwinkle,12
2,Lopez,Liam,blue,green,13
3,Lee,Isabella,,11
4,Fisher,Mason,gray,-1
5,Gupta,Olivia,9,102
6,,Robinson,,Sophia,,blue,,12



In a friendly way, we have a header line that indicates reasonable field names and provides a hint as to the meaning of each column.  Programmatically, we may not wish to work with the punctuation marks and spaces inside some field names, but that is a matter of tool convenience that we can address with the APIs that data processing tools give us.

Let us think about each record in turn.  Mia Johnson, student 1, seems to have a problem-free record.  Her row has five values separated by four commas, and each data value meets our intuitive expectations about the data type and value domain.  The problems start hereafter.

Liam Lopez has too many fields in his row.  However, both columns 4 and 5 seem clearly to be in the lexicon of color names.  Perhaps a duplicate entry occurred or the compound color "blue-green" was intended.  Structurally the row has issues, but several plausible remediations suggest themselves.

Isabella Lee is perhaps no problem at all.  One of her fields is empty, meaning no favorite color is available.  But structurally, this row is perfectly fine for CSV format.  We will need to use some domain or problem knowledge to decide how to handle the missing value.

Mason Fisher is perhaps similar to Isabella.  The recorded age of -1 makes no sense in the nature of "age" as a data field, at least as we usually understand it (but maybe the encoding intends something different).  On the other hand, -1 is one of several placeholder values used very commonly to represent missing data.  We need to know our specific problem to know whether we can process the data with a missing age, but many times we can handle that.  However, we still need to be careful not to treat the -1 as a plain value; for example, the mean, minimum, or standard deviation of ages might be thrown off by that.

Olivia Gupta starts to present a trickier problem.  Structurally her row looks perfect.  But '9' is probably not a string in our lexicon of color names.  And under our understanding of the data concerning a 6th grade class, we don't expect 102 year old students to be in it.  To solve this row, we really need to know more about the collection procedure and the intention of the data.  Perhaps a separate mapping of numbers to colors exists somewhere.  Perhaps an age of 12 was mistranscribed as 102; but also perhaps a 102 year old serves as a teaching assistant in this class and not only students are recorded.

Sophia Robinson returns us to what looks like an obvious structural error.  The row, upon visual inspection, contains perfectly good and plausible values, but they are separated by duplicate commas.  Somehow, persumably, a mechanical error resulted in the line being formatted wrongly.  However, most high-level tools are likely to choke on the row in an uninformative way, and we will probably need to remediate the issue more manually.

We have a pretty good idea what to do with these six rows of data, and even re-entering them from scratch would not be difficult.  If we had a million rows instead, the difficulty would grow greatly, and would require considerable effort before we arrived at usable data.

## Nomenclature

```
| Field-like  | Record-like 
|-------------|-------------
| Feature     | Row
| Measurement | Observation
| Column      | Sample
| Field       | Record
| Variable    |
```

In this course the terms *feature*, *field*, *measurement*, *column*, and occasionally *variable* amore-or-less interchangeably.  Likewise, the terms *row*, *record*, *observation*, and *sample* are also near synonyms.  *Tuple* is used for the same concept when discussing databases (especially academically). In different academic or business fields, different ones of these terms are more prominent; and likewise different software tools choose among these. 

Conceptually, most data can be thought of as a number of occasions on which we measure various attributes of a common underlying *thing*.  In most tools, it is usually convenient to put these observations/samples each in a row; and correspondingly to store each of the measurements/features/fields pertaining to that thing in a column containing corresponding data for other comparable *things*.

Inasmuch as I vary the use of these roughly equivalent terms, it is simply better to fit with the domain under discussion and to make readers familiar with all the terms, which they are likely to encounter in various places for a similar intention.  The choice among near synonyms is also guided by the predominant use within the particular tool, library, or programming community that is currently being discussed.

There are many tools which you are fairly likely to encounter during your work. It is valuable for readers to understand the general role of tools they may require in their specific tasks.  When mentioning tools, I try to provide a general conceptual framework for what *kind* of thing that tool is, but you certainly do not need to be familiar with most of the tools mentioned.

## Visual Rendering

Data is often presented with the Polars or Pandas DataFrame libraries.

In [2]:
import sys
import polars as pl
import pandas as pd
print("Python", sys.version)
print("Polars", pl.__version__)
print("Pandas", pd.__version__)

Python 3.13.0 (main, Oct 16 2024, 03:23:02) [Clang 18.1.8 ]
Polars 1.16.0
Pandas 2.2.3


Visualizing a particular data set might look like the table shown.

In [3]:
cleaned

Student_No,Last_Name,First_Name,Favorite_Color,Age
i64,str,str,str,str
1,"""Johnson""","""Mia""","""periwinkle""","""12"""
2,"""Lopez""","""Liam""","""blue-green""","""13"""
3,"""Lee""","""Isabella""","""<missing>""","""11"""
4,"""Fisher""","""Mason""","""gray""","""N/A"""
5,"""Gupta""","""Olivia""","""sepia""","""N/A"""
6,"""Robinson""","""Sophia""","""blue""","""12"""


The data sets and supporting materials for this book are available for download. These extra materials will allow you to try out variations on the code shown and discussed, perhaps using different libraries or programming languages.

Throughout this course I am *strongly opinionated* about a number of technical questions.  I do not believe it will be difficult to distinguish my opinions from the *mere facts* I also present.  I have worked in this area for a number of years, and I hope to share with readers the conclusions I have reached.  If you disagree with claims I make, I hope you will benefit both from what you learn anew and what you are able to reformulate in strengthening your own opinions and conclusions.

* <a href="https://greenteapress.com/thinkstats2/thinkstats2.pdf"><u>Think Stats: Exploratory Data Analysis in Python</u></a>, Allen B. Downey, 2014 (O'Reilly Media; available both in free PDF and HTML versions, and as a printed book).
* <u>All of Statistics: A Concise Course in Statistical Inference</u>, Larry Wasserman, 2004 (Springer).
* <a href="https://socviz.co/index.html"><u>Data Visualization: A practical introduction</u></a>, Kieran Healy, 2018 (Princeton University Press; a non-final draft is available free online).
* <a href="https://www.edwardtufte.com/tufte/books_be"><u>The Visual Display of Quantitative Information</u></a>, Edward Tufte, 2001 (Graphics Press; all four of Tufte's visualization books are canonical in the field).

This course does not use mathematics and statistics heavily, but the references shown are worth exploring.  We also do not focused on the *ethics of data visualization*, but I have tried to be conscientious in using plots, which I use throughout the text.  Some good texts considering these issues are listed.

## Data Hygiene

If a data set is of moderate size, versioning within program flow is a good choice.

```python
data1 = read_format(raw_data)
data2 = transformation_1(data1)
data3 = transformation_2(data2)
# ... etc ...
```

When you use any version, anywhere else in a large program, it is clear from the variable name (or lookup key, etc.) which version is involved, and problems can be more easily diagnosed.

It is crucial to good practice of data science to *version* data sets as we work with them.  When we draw some conclusion, or even simply when we prepare for the next transformation step, it is important to indicate which version of the data this action is based on.  There are several different ways in which data sets may be versioned.  

### Large Data

```python
data = Annotated(read_format(raw_data))
inplace_transform_1(data)
data.steps += "Transformed by step 1"
# ... actions on data ...
inplace_transform_2(data)
data.steps += "Transformed by step 2"
# ... etc ...
```

If a data set is somewhat larger in size—to the point where keeping a number of near-copies in memory is a resource constraint—it is possible instead to track changes simply as metadata on the working data set.  This does not allow simultaneous access to multiple versions in code, but is still very useful for debugging and analysis. 

For transformations that you wish to persist longer than the run of a single program, use of version control systems (VCS) is highly desirable.  Most VCSs allow a concept of a *branch* where different versions of files can be maintained in parallel.  Even if your data set versions are strictly linear, it is possible to revert to a particular earlier version if necessary.  Using accurate and descriptive commit messages is a great benefit to data versioning.